In [4]:
from pathlib import Path
import cv2
import numpy as np

In [5]:
calibration_images_path = Path.cwd() / Path('tests') / Path('images')
calibration_images = tuple(calibration_images_path.glob('left*'))
calibration_images[5]

PosixPath('/home/jsk/GitHub/cv2stuff/cv2stuff/tests/images/left12.jpg')

In [6]:
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
criteria

(3, 30, 0.001)

In [7]:
pattern_dimensions = (9, 6) # The inner corners of the sample chessboard image.
dimensions_product = pattern_dimensions[0] * pattern_dimensions[1]

In [8]:
objp = np.zeros((dimensions_product,3), np.float32) # empty array
objp[:,:2] = np.mgrid[0:pattern_dimensions[0],0:pattern_dimensions[1]].T.reshape(-1,2) # index the array

In [9]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

In [10]:
fname = str(calibration_images[0])

In [11]:
img = cv2.imread(fname)
gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [12]:
ret, corners = cv2.findChessboardCorners(gray, pattern_dimensions, None)
ret

True

In [13]:
objpoints.append(objp)

In [14]:
cv2.cornerSubPix(gray, corners,(11,11), (-1,-1), criteria)

array([[[ 188.52177429,  130.59634399]],

       [[ 223.25431824,  127.10599518]],

       [[ 260.33712769,  123.72382355]],

       [[ 299.31347656,  120.21031189]],

       [[ 340.14273071,  117.28233337]],

       [[ 382.4118042 ,  114.39073181]],

       [[ 425.84460449,  112.19089508]],

       [[ 470.29394531,  110.21006012]],

       [[ 514.56268311,  109.15750885]],

       [[ 185.58255005,  168.31446838]],

       [[ 220.92555237,  165.8598175 ]],

       [[ 258.52218628,  163.19372559]],

       [[ 298.12036133,  160.62052917]],

       [[ 339.50344849,  158.407547  ]],

       [[ 382.67443848,  156.35827637]],

       [[ 427.01651001,  154.53375244]],

       [[ 472.39154053,  153.32444763]],

       [[ 517.5065918 ,  152.16775513]],

       [[ 183.30487061,  207.50357056]],

       [[ 218.83789062,  205.89527893]],

       [[ 256.93572998,  204.27267456]],

       [[ 297.06646729,  202.78793335]],

       [[ 338.93032837,  201.50622559]],

       [[ 382.76010132,  200.21937

In [15]:
imgpoints.append(corners)

In [16]:
cv2.drawChessboardCorners(img, pattern_dimensions, corners, ret)

array([[[ 0,  0,  0],
        [ 0,  0,  0],
        [ 1,  1,  1],
        ..., 
        [ 0,  0,  0],
        [ 5,  5,  5],
        [ 0,  0,  0]],

       [[ 0,  0,  0],
        [ 1,  1,  1],
        [ 3,  3,  3],
        ..., 
        [ 6,  6,  6],
        [12, 12, 12],
        [ 1,  1,  1]],

       [[ 2,  2,  2],
        [ 3,  3,  3],
        [ 5,  5,  5],
        ..., 
        [10, 10, 10],
        [14, 14, 14],
        [ 0,  0,  0]],

       ..., 
       [[10, 10, 10],
        [37, 37, 37],
        [41, 41, 41],
        ..., 
        [61, 61, 61],
        [48, 48, 48],
        [ 6,  6,  6]],

       [[ 8,  8,  8],
        [34, 34, 34],
        [35, 35, 35],
        ..., 
        [43, 43, 43],
        [41, 41, 41],
        [ 8,  8,  8]],

       [[ 0,  0,  0],
        [11, 11, 11],
        [10, 10, 10],
        ..., 
        [ 9,  9,  9],
        [13, 13, 13],
        [ 0,  0,  0]]], dtype=uint8)

In [17]:
cv2.imshow('img',img)
cv2.waitKey(500)
cv2.destroyAllWindows()

In [18]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [19]:
fname = str(calibration_images[5])

In [20]:
img = cv2.imread(fname)

In [21]:
h,  w = img.shape[:2]

In [22]:
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))
newcameramtx, roi

(array([[ 146.21296692,    0.        ,  492.43311094],
        [   0.        ,  133.51545715,  397.62325063],
        [   0.        ,    0.        ,    1.        ]]), (0, 0, 0, 0))

In [23]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)

In [24]:
# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]
cal_image = str(calibration_images_path) + '/' + 'calibresult.png'
written = cv2.imwrite(cal_image, dst)
roi, dst, cal_image

((0, 0, 0, 0),
 array([], shape=(0, 0, 3), dtype=uint8),
 '/home/jsk/GitHub/cv2stuff/cv2stuff/tests/images/calibresult.png')

In [25]:
# undistort
mapx,mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
mapx, mapy

(None, None)

In [26]:
dst = cv2.remap(img,mapx,mapy,cv2.INTER_LINEAR)

error: /home/jsk/GitHub/opencv/modules/imgproc/src/imgwarp.cpp:4672: error: (-215) _map1.size().area() > 0 in function remap
